## Overfitting and validation homework

In this homework, we will cover how to implement the regularization techniques we learned in class:
- Early Stopping
- Weight Decay (L2 Regularization)
- Dropout
- Data Augmentation

In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.datasets import mnist, cifar10
from keras import regularizers
import matplotlib.pyplot as plt

from numpy.random import seed
seed(1)

/Users/aqua/miniconda3/envs/ai4all/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def create_model():
    """ Creates a new convolutional neural network and returns it """
    seed(1)
    input_shape = (32, 32, 3)
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    # initiate RMSprop optimizer; slightly better than sgd; account for momentum (how big previous steps were)
    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
    # Let's train the model using RMSprop
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

### Image preprocessing

This code block loads our dataset and does some initial preprocessing. We are using the `MNIST` dataset of hand written digits. You can read about MNIST here: http://yann.lecun.com/exdb/mnist/

In [3]:
(x_train, y_train), (x_val, y_val) = cifar10.load_data()
batch_size = 128
num_classes = 10
epochs = 150

# input image dimensions
img_rows, img_cols = 32, 32

num_train = 1000
num_val = 500  # We choose such a large validation set to illustrate the effects of overfitting without noise

x_train = x_train[:num_train]
y_train = y_train[:num_train]
x_val = x_val[:num_val]
y_val = y_val[:num_val]

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_train /= 255
x_val /= 255

print('x_train shape:', x_train.shape)
print('x_val shape:', x_val.shape)
print(x_train.shape[0], 'train samples')
print(x_val.shape[0], 'validation samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)

# This function will print the training
def evaluate_acc(model):
    train_score = model.evaluate(x_train, y_train, verbose=0)
    print('Train loss:', train_score[0])
    print('Train accuracy:', train_score[1])
    val_score = model.evaluate(x_val, y_val, verbose=0)
    print('Validation loss:', val_score[0])
    print('Validation accuracy:', val_score[1])

x_train shape: (1000, 32, 32, 3)
x_val shape: (500, 32, 32, 3)
1000 train samples
500 validation samples


### No regularization
First, lets just try seeing how the model performs with no regularization. We are going to train for 150 epochs on 

This should take around 3-5 minutes to train. Go take a break and grab a snack!

In [4]:
model = create_model()
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_val, y_val),
          shuffle=True)
evaluate_acc(model)

W0708 21:31:14.892477 4610237888 deprecation_wrapper.py:119] From /Users/aqua/miniconda3/envs/ai4all/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0708 21:31:14.906038 4610237888 deprecation_wrapper.py:119] From /Users/aqua/miniconda3/envs/ai4all/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0708 21:31:14.908219 4610237888 deprecation_wrapper.py:119] From /Users/aqua/miniconda3/envs/ai4all/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0708 21:31:14.933391 4610237888 deprecation_wrapper.py:119] From /Users/aqua/miniconda3/envs/ai4all/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.

Train on 1000 samples, validate on 500 samples
Epoch 1/150
1000/1000 [==============================] - 2s 2ms/step - loss: 2.2861 - acc: 0.1330 - val_loss: 2.2651 - val_acc: 0.1580
Epoch 2/150
1000/1000 [==============================] - 1s 1ms/step - loss: 2.2382 - acc: 0.1990 - val_loss: 2.2345 - val_acc: 0.2160
Epoch 3/150
1000/1000 [==============================] - 1s 1ms/step - loss: 2.1785 - acc: 0.2420 - val_loss: 2.1677 - val_acc: 0.1680
Epoch 4/150
1000/1000 [==============================] - 1s 1ms/step - loss: 2.1002 - acc: 0.2950 - val_loss: 2.0860 - val_acc: 0.2740
Epoch 5/150
1000/1000 [==============================] - 1s 1ms/step - loss: 2.0195 - acc: 0.3090 - val_loss: 2.0127 - val_acc: 0.3180
Epoch 6/150
1000/1000 [==============================] - 1s 1ms/step - loss: 1.9370 - acc: 0.3610 - val_loss: 1.9840 - val_acc: 0.3100
Epoch 7/150
1000/1000 [==============================] - 1s 1ms/step - loss: 1.8933 - acc: 0.3610 - val_loss: 1.9680 - val_acc: 0.2780
Epoch 8/

Epoch 61/150
1000/1000 [==============================] - 2s 2ms/step - loss: 1.1996 - acc: 0.5890 - val_loss: 1.7577 - val_acc: 0.3620
Epoch 62/150
1000/1000 [==============================] - 1s 1ms/step - loss: 1.0959 - acc: 0.6560 - val_loss: 1.7778 - val_acc: 0.3860
Epoch 63/150
1000/1000 [==============================] - 1s 1ms/step - loss: 1.1507 - acc: 0.6380 - val_loss: 1.8218 - val_acc: 0.3760
Epoch 64/150
1000/1000 [==============================] - 1s 1ms/step - loss: 1.0862 - acc: 0.6370 - val_loss: 1.7529 - val_acc: 0.3900
Epoch 65/150
1000/1000 [==============================] - 1s 1ms/step - loss: 1.1436 - acc: 0.6230 - val_loss: 1.7548 - val_acc: 0.3920
Epoch 66/150
1000/1000 [==============================] - 1s 1ms/step - loss: 1.0877 - acc: 0.6340 - val_loss: 1.7877 - val_acc: 0.3980
Epoch 67/150
1000/1000 [==============================] - 1s 1ms/step - loss: 1.0758 - acc: 0.6550 - val_loss: 1.7523 - val_acc: 0.3900
Epoch 68/150
1000/1000 [========================

1000/1000 [==============================] - 1s 1ms/step - loss: 0.5453 - acc: 0.8520 - val_loss: 2.1208 - val_acc: 0.3780
Epoch 122/150
1000/1000 [==============================] - 1s 1ms/step - loss: 0.5609 - acc: 0.8320 - val_loss: 2.0381 - val_acc: 0.3840
Epoch 123/150
1000/1000 [==============================] - 1s 1ms/step - loss: 0.4852 - acc: 0.8730 - val_loss: 2.0601 - val_acc: 0.4020
Epoch 124/150
1000/1000 [==============================] - 1s 1ms/step - loss: 0.5219 - acc: 0.8590 - val_loss: 2.0212 - val_acc: 0.3840
Epoch 125/150
1000/1000 [==============================] - 1s 1ms/step - loss: 0.4886 - acc: 0.8590 - val_loss: 2.1526 - val_acc: 0.3880
Epoch 126/150
1000/1000 [==============================] - 1s 1ms/step - loss: 0.4526 - acc: 0.8900 - val_loss: 2.1624 - val_acc: 0.3760
Epoch 127/150
1000/1000 [==============================] - 1s 1ms/step - loss: 0.5142 - acc: 0.8440 - val_loss: 2.0603 - val_acc: 0.3840
Epoch 128/150
1000/1000 [==============================

### Early Stopping
You should get training accuracy around 0.783, and validation accuracy around 0.37. This is a pretty big gap, one which we cannot improve with just training. Let's first try to add early stopping!

Hints: 
Check out how EarlyStopping is implemented in Keras and how you would go about adding it to your model here: https://keras.io/callbacks/.
You may actually get worse performance initially by adding EarlyStopping. Try tuning the `patience` argument (read up on it in the keras docs). However, while it makes training faster, you likely won't see much improvement in validation accuracy (I get around 2% increase in validation accuracy). Note that while training accuracy may be lower, your validation accuracy will be higher. This is because the model is not super overfit to the training data yet, and has learned some understandable features.

In [5]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience=10)
model = create_model()
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_val, y_val),
          shuffle=True,
          callbacks=[early_stopping])
evaluate_acc(model)

Train on 1000 samples, validate on 500 samples
Epoch 1/150
1000/1000 [==============================] - 2s 2ms/step - loss: 2.2860 - acc: 0.1320 - val_loss: 2.2646 - val_acc: 0.1600
Epoch 2/150
1000/1000 [==============================] - 1s 1ms/step - loss: 2.2374 - acc: 0.2040 - val_loss: 2.2340 - val_acc: 0.2180
Epoch 3/150
1000/1000 [==============================] - 1s 1ms/step - loss: 2.1774 - acc: 0.2500 - val_loss: 2.1655 - val_acc: 0.1680
Epoch 4/150
1000/1000 [==============================] - 1s 1ms/step - loss: 2.0975 - acc: 0.2970 - val_loss: 2.0839 - val_acc: 0.2840
Epoch 5/150
1000/1000 [==============================] - 1s 1ms/step - loss: 2.0166 - acc: 0.3160 - val_loss: 2.0115 - val_acc: 0.3280
Epoch 6/150
1000/1000 [==============================] - 1s 1ms/step - loss: 1.9353 - acc: 0.3540 - val_loss: 1.9824 - val_acc: 0.3080
Epoch 7/150
1000/1000 [==============================] - 2s 2ms/step - loss: 1.8924 - acc: 0.3540 - val_loss: 1.9677 - val_acc: 0.2780
Epoch 8/

### Weight Decay

Early stopping has helped us not train longer than we need to, but in this case doesn't help much with validation accuracy. This means that the gap between validation and training accuracy is always large, and there are never any "peaky" moments where the distance is decreased. Let's next try to add L2 Weight Decay to our model.

Hint: As example code, you can use the following code to declare a Dense layer with 64 hidden neurons and l2 regularization with weight decay strength 0.01. You can add this to *all* of the *learnable* layers in the model in `create_model`. Check out the documentation on regularizers here: https://keras.io/regularizers/

```
from keras import regularizers

Dense(64, input_dim=64, kernel_regularizer=regularizers.l2(0.01))
```

Add it now to the model in `create_model`, and retrain it here to see your new results. Note that you can tweak your weight decay strength as a hyperparameter. However, I have personally done this and found best results with `0.01`. Can you see if you can outperform this? I personally see about a 1% improvement, and a much smaller gap between training accuracy and validation accuracy. This is due to the fact that while we are less overfit to our training data, we still learning valuable features of the images.

In [8]:
def create_model_weight_decay():
    """ Creates a new convolutional neural network with weight decay and returns it """
    seed(1)
    input_shape = (32, 32, 3)
    reg = regularizers.l2(0.01)
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:], kernel_regularizer=reg))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3), kernel_regularizer=reg))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=reg))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3), kernel_regularizer=reg))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(512, kernel_regularizer=reg))
    model.add(Activation('relu'))
    model.add(Dense(num_classes, kernel_regularizer=reg))
    model.add(Activation('softmax'))

    # initiate RMSprop optimizer
    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
    # Let's train the model using RMSprop
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

In [7]:
early_stopping = EarlyStopping(patience=10)
model = create_model_weight_decay()
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_val, y_val),
          shuffle=True,
          callbacks=[early_stopping])
evaluate_acc(model)

Train on 1000 samples, validate on 500 samples
Epoch 1/150
1000/1000 [==============================] - 2s 2ms/step - loss: 11.8857 - acc: 0.1320 - val_loss: 11.4827 - val_acc: 0.1160
Epoch 2/150
1000/1000 [==============================] - 2s 2ms/step - loss: 11.2608 - acc: 0.1440 - val_loss: 10.9917 - val_acc: 0.1120
Epoch 3/150
1000/1000 [==============================] - 2s 2ms/step - loss: 10.7982 - acc: 0.1580 - val_loss: 10.5613 - val_acc: 0.1280
Epoch 4/150
1000/1000 [==============================] - 2s 2ms/step - loss: 10.3837 - acc: 0.1780 - val_loss: 10.1643 - val_acc: 0.1620
Epoch 5/150
1000/1000 [==============================] - 2s 2ms/step - loss: 9.9938 - acc: 0.1910 - val_loss: 9.7863 - val_acc: 0.1680
Epoch 6/150
1000/1000 [==============================] - 2s 2ms/step - loss: 9.6173 - acc: 0.2200 - val_loss: 9.4298 - val_acc: 0.2680
Epoch 7/150
1000/1000 [==============================] - 2s 2ms/step - loss: 9.2584 - acc: 0.2420 - val_loss: 9.0785 - val_acc: 0.2360


Epoch 61/150
1000/1000 [==============================] - 2s 2ms/step - loss: 3.2799 - acc: 0.4430 - val_loss: 3.5321 - val_acc: 0.3420
Epoch 62/150
1000/1000 [==============================] - 1s 1ms/step - loss: 3.2115 - acc: 0.4600 - val_loss: 3.4985 - val_acc: 0.3600
Epoch 63/150
1000/1000 [==============================] - 1s 1ms/step - loss: 3.2129 - acc: 0.4630 - val_loss: 3.5717 - val_acc: 0.3280
Epoch 64/150
1000/1000 [==============================] - 1s 1ms/step - loss: 3.1773 - acc: 0.4790 - val_loss: 3.4323 - val_acc: 0.3700
Epoch 65/150
1000/1000 [==============================] - 1s 1ms/step - loss: 3.1924 - acc: 0.4730 - val_loss: 3.4356 - val_acc: 0.3600
Epoch 66/150
1000/1000 [==============================] - 1s 1ms/step - loss: 3.1383 - acc: 0.4640 - val_loss: 3.4480 - val_acc: 0.3760
Epoch 67/150
1000/1000 [==============================] - 1s 1ms/step - loss: 3.1028 - acc: 0.4710 - val_loss: 3.4105 - val_acc: 0.3660
Epoch 68/150
1000/1000 [========================

1000/1000 [==============================] - 1s 1ms/step - loss: 2.3669 - acc: 0.6130 - val_loss: 2.9581 - val_acc: 0.3780
Epoch 122/150
1000/1000 [==============================] - 1s 1ms/step - loss: 2.3547 - acc: 0.6080 - val_loss: 2.9729 - val_acc: 0.3620
Epoch 123/150
1000/1000 [==============================] - 1s 1ms/step - loss: 2.3701 - acc: 0.6020 - val_loss: 2.9279 - val_acc: 0.4040
Epoch 124/150
1000/1000 [==============================] - 1s 1ms/step - loss: 2.3409 - acc: 0.6040 - val_loss: 2.9100 - val_acc: 0.3920
Epoch 125/150
1000/1000 [==============================] - 1s 1ms/step - loss: 2.3325 - acc: 0.5990 - val_loss: 3.0096 - val_acc: 0.3940
Epoch 126/150
1000/1000 [==============================] - 1s 1ms/step - loss: 2.3571 - acc: 0.5940 - val_loss: 2.9246 - val_acc: 0.3960
Epoch 127/150
1000/1000 [==============================] - 1s 1ms/step - loss: 2.2840 - acc: 0.6190 - val_loss: 2.9098 - val_acc: 0.3800
Epoch 128/150
1000/1000 [==============================

### Dropout
We next add dropout layers to our model.

In [9]:
def create_model_dropout():
    """ Creates a new convolutional neural network with weight decay and returns it """
    seed(1)
    input_shape = (32, 32, 3)
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    # initiate RMSprop optimizer
    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
    # Let's train the model using RMSprop
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [10]:
early_stopping = EarlyStopping(patience=10)
model = create_model_dropout()
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_val, y_val),
          shuffle=True,
          callbacks=[early_stopping])
evaluate_acc(model)

W0708 21:43:59.592664 4610237888 deprecation.py:506] From /Users/aqua/miniconda3/envs/ai4all/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Train on 1000 samples, validate on 500 samples
Epoch 1/150
1000/1000 [==============================] - 2s 2ms/step - loss: 2.3047 - acc: 0.0940 - val_loss: 2.2880 - val_acc: 0.1500
Epoch 2/150
1000/1000 [==============================] - 2s 2ms/step - loss: 2.2852 - acc: 0.1360 - val_loss: 2.2775 - val_acc: 0.1200
Epoch 3/150
1000/1000 [==============================] - 2s 2ms/step - loss: 2.2778 - acc: 0.1310 - val_loss: 2.2637 - val_acc: 0.1220
Epoch 4/150
1000/1000 [==============================] - 2s 2ms/step - loss: 2.2624 - acc: 0.1460 - val_loss: 2.2439 - val_acc: 0.1280
Epoch 5/150
1000/1000 [==============================] - 2s 2ms/step - loss: 2.2383 - acc: 0.1670 - val_loss: 2.2194 - val_acc: 0.2300
Epoch 6/150
1000/1000 [==============================] - 2s 2ms/step - loss: 2.1961 - acc: 0.1890 - val_loss: 2.1738 - val_acc: 0.2240
Epoch 7/150
1000/1000 [==============================] - 2s 2ms/step - loss: 2.1412 - acc: 0.2310 - val_loss: 2.1269 - val_acc: 0.2680
Epoch 8/

Epoch 61/150
1000/1000 [==============================] - 2s 2ms/step - loss: 1.4888 - acc: 0.4510 - val_loss: 1.7687 - val_acc: 0.3600
Epoch 62/150
1000/1000 [==============================] - 2s 2ms/step - loss: 1.5099 - acc: 0.4550 - val_loss: 1.7112 - val_acc: 0.3740
Epoch 63/150
1000/1000 [==============================] - 2s 2ms/step - loss: 1.4882 - acc: 0.4600 - val_loss: 1.7342 - val_acc: 0.3800
Epoch 64/150
1000/1000 [==============================] - 2s 2ms/step - loss: 1.4904 - acc: 0.4500 - val_loss: 1.7081 - val_acc: 0.3700
Epoch 65/150
1000/1000 [==============================] - 2s 2ms/step - loss: 1.4703 - acc: 0.4890 - val_loss: 1.7119 - val_acc: 0.3860
Epoch 66/150
1000/1000 [==============================] - 2s 2ms/step - loss: 1.4544 - acc: 0.4770 - val_loss: 1.7579 - val_acc: 0.3680
Epoch 67/150
1000/1000 [==============================] - 2s 2ms/step - loss: 1.4470 - acc: 0.4700 - val_loss: 1.7096 - val_acc: 0.3820
Epoch 68/150
1000/1000 [========================

I get around 41% validation accuracy using dropout.